In [ ]:
import cntk as C
import numpy as np
from sklearn.preprocessing import scale

# Building Neural Network with Diabetes data

## Loading dataset

In [ ]:
dataset = np.loadtxt("../build neural networks/diabetes.csv", delimiter=",", skiprows=1 )
diabetes_X = dataset[:,0:8]
diabetes_Y = dataset[:,8]
diabetes_X = scale(diabetes_X,axis=0)

## Building the network

In [ ]:
input_dim = 8
num_output_classes = 1

In [ ]:
# Helper function to generate a random data sample
def generate_random_data_sample(sample_size, feature_dim, num_classes):    
    idx = np.random.choice(diabetes_X.shape[0], size=(sample_size, 1), replace=False)
    X = diabetes_X[idx].astype(np.float32)
    Y = diabetes_Y[idx]

    # converting class 0 into the vector "1 0 0",
    # class 1 into vector "0 1 0", ...
    class_ind = [Y==class_number for class_number in range(num_classes)]
    Y = np.asarray(np.hstack(class_ind), dtype=np.float32)
    return X, Y

In [ ]:
mysamplesize = 64
features, labels = generate_random_data_sample(mysamplesize, input_dim, num_output_classes)

num_hidden_layers = 1
hidden_layers_dim = 8

input = C.input_variable(input_dim)
label = C.input_variable(num_output_classes)

In [ ]:
def create_model(features):
    with C.layers.default_options(init=C.layers.glorot_uniform(), activation=C.sigmoid):
        h = features
        for _ in range(num_hidden_layers):
            h = C.layers.Dense(hidden_layers_dim, activation=C.sigmoid)(h)
        last_layer = C.layers.Dense(num_output_classes, activation = None)

        return last_layer(h)

In [ ]:
model = create_model(input)

## Training the network

In [ ]:
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose:
            print ("Minibatch: {}, Train Loss: {}, Train Error: {}".format(mb, training_loss, eval_error))

    return mb, training_loss, eval_error

In [ ]:
loss = C.cross_entropy_with_softmax(model, label)
eval_error = C.classification_error(model, label)

In [ ]:
learning_rate = 0.5
lr_schedule = C.learning_parameter_schedule(learning_rate)
learner = C.sgd(model.parameters, lr_schedule)
trainer = C.Trainer(model, (loss, eval_error), [learner])

In [ ]:
minibatch_size = 10
num_minibatches_to_train = 300

training_progress_output_freq = 20
plotdata = {"batchsize":[], "loss":[], "error":[]}

In [ ]:
for i in range(0, int(num_minibatches_to_train)):    
    features, labels = generate_random_data_sample(minibatch_size, input_dim, num_output_classes)

    # Specify the input variables mapping in the model to actual minibatch data for training
    trainer.train_minibatch({input : features, label : labels})
    batchsize, loss, error = print_training_progress(trainer, i,
                                                     training_progress_output_freq, verbose=0)

    if not (loss == "NA" or error =="NA"):
        plotdata["batchsize"].append(batchsize)
        plotdata["loss"].append(loss)
        plotdata["error"].append(error)

In [ ]:
# Generate new data
test_minibatch_size = 10
#features, labels = generate_random_data_sample(test_minibatch_size, input_dim, num_output_classes)
features, labels = generate_random_data_sample(test_minibatch_size, input_dim, num_output_classes)
trainer.test_minibatch({input : features, label : labels})

out = C.softmax(model)

predicted_label_probs = out.eval({input : features})

print("Label    :", [np.argmax(label) for label in labels])
print("Predicted:", [np.argmax(row) for row in predicted_label_probs])

## Prediction 

In [ ]:
test_data = np.array([6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0], dtype='float32')

In [ ]:
predicted_label_probs = model.eval(test_data)
print("Predicted:", [np.argmax(row) for row in predicted_label_probs])

## Save model

In [ ]:
model.save('cntk_diabetes.model')

## Load model

In [ ]:
from cntk.ops.functions import load_model

z = load_model("cntk_diabetes.model")

## Read data from loaded model

### Read weights and biases

In [ ]:
parameters = z.parameters
for parameter in parameters:
    print(parameter.name, parameter.shape, "\n", parameter.value) 

### Read structure of network

In [ ]:
layers = C.logging.graph.depth_first_search(
            model, (lambda x : type(x) == C.Function and x.is_block) , depth = 0)
for layer in layers:
    print(layer.op_name) #Typ des Layers z.B. Dense
    print(layer.is_composite)
    print(layer.arguments) 
    print(layer.constants)
    print(layer.parameters)
    print(layer.attributes)
    print('-----')